In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import statsmodels.api as sm

data_path = Path("data")

# Read the 'Monthly' sheet from the median_house_price file
prices = pd.read_excel(data_path / "median_house_price.xlsx", sheet_name="Monthly")
# Ensure the date column is datetime
if not pd.api.types.is_datetime64_any_dtype(prices["observation_date"]):
    prices["observation_date"] = pd.to_datetime(prices["observation_date"])

# FIX: Actually set the index properly
prices = prices.set_index("observation_date")

# Read CPI file and parse dates; adjust column name if different
cpi_data = pd.read_excel(
    data_path / "cpi.xlsx",
    sheet_name="Monthly",
    parse_dates=["observation_date"]
)

# Ensure the date column is datetime
if not pd.api.types.is_datetime64_any_dtype(cpi_data["observation_date"]):
    cpi_data["observation_date"] = pd.to_datetime(cpi_data["observation_date"])

# Filter CPI data to start from 1963-01-01 (inclusive)
cpi_data = cpi_data[cpi_data["observation_date"] >= "1963-01-01"]

# Extract CPI series and set index
cpi = cpi_data.set_index("observation_date")["cpi"]

# Quick check
print(cpi.head())
print(f"Rows after filtering: {len(cpi)}")

# Calculate real house prices
cpi_100 = cpi / 100  # Convert CPI to index form
real_prices = prices.price / cpi_100  # Now both are properly indexed

# Read other datasets similarly and compute their monthly differences
loans = pd.read_excel(data_path / "loans.xlsx", sheet_name="Monthly")
loans = loans.set_index("observation_date")
log_loans = np.log(loans.loans)
log_loans = log_loans.to_frame(name='loans').set_index(loans.index)



net_wealth = pd.read_excel(data_path / "net_wealth_of_top_1.xlsx", sheet_name="Quarterly")
net_wealth = net_wealth.set_index("observation_date")
net_wealth.rename(columns={"percentage": "net_wealth"}, inplace=True)
net_wealth_monthly = net_wealth.resample('MS').interpolate(method='linear').ffill().bfill()

vacancy_rate = pd.read_excel(data_path / "vacancy_rate.xlsx", sheet_name="Quarterly")
vacancy_rate = vacancy_rate.set_index("observation_date")
vacancy_rate_monthly = vacancy_rate.resample('MS').interpolate(method='linear').ffill().bfill()

mortgage_rate = pd.read_excel(data_path / "MORTGAGE30US.xlsx", sheet_name="Weekly, Ending Thursday")
mortgage_rate = mortgage_rate.set_index("observation_date")
mortgage_rate.rename(columns={"MORTGAGE30US": "mortgage_rate"}, inplace=True)
mortgage_rate_monthly = mortgage_rate.resample('MS').interpolate(method='linear').ffill().bfill()

## Import more data
pca_path = Path("data/PCA")

active_listings = pd.read_excel(pca_path / "ACTLISCOUUS.xlsx", sheet_name="Monthly")
active_listings.rename(columns={"ACTLISCOUUS": "active_listings"}, inplace=True)
active_listings.set_index("observation_date", inplace=True)

median_house_income = pd.read_excel(pca_path / "MEHOINUSA672N.xlsx", sheet_name="Annual")
median_house_income['observation_date'] = pd.to_datetime(median_house_income['observation_date'], errors='coerce')
median_house_income.set_index('observation_date', inplace=True)
median_house_income.rename(columns={"MEHOINUSA672N": "median_house_income"}, inplace=True)
median_house_income_monthly = median_house_income.resample('MS').interpolate(method='linear').ffill().bfill()

monthly_supply_homes = pd.read_excel(pca_path / "MSACSR.xlsx", sheet_name="Monthly")
monthly_supply_homes.set_index("observation_date", inplace=True)
monthly_supply_homes.rename(columns={"MSACSR": "monthly_supply_homes"}, inplace=True)

labor_share = pd.read_excel(pca_path / "PRS85006173.xlsx", sheet_name="Quarterly")
labor_share.set_index("observation_date", inplace=True)
labor_share.rename(columns={"PRS85006173": "labor_share"}, inplace=True)
labor_share_monthly = labor_share.resample('MS').interpolate(method='linear').ffill().bfill()

unemployment_rate = pd.read_excel(pca_path / "UNRATE.xlsx", sheet_name="Monthly")
unemployment_rate.set_index("observation_date", inplace=True)
unemployment_rate.rename(columns={"UNRATE": "unemployment_rate"}, inplace=True)

share_net_worth_bottom_50 = pd.read_excel(pca_path / "WFRBSB50215.xlsx", sheet_name="Quarterly")
share_net_worth_bottom_50.set_index("observation_date", inplace=True)
share_net_worth_bottom_50.rename(columns={"WFRBSB50215": "share_net_worth_bottom_50"}, inplace=True)
share_net_worth_bottom_50_monthly = share_net_worth_bottom_50.resample('MS').interpolate(method='linear').ffill().bfill()
log_share_net_worth_bottom_50_monthly = np.log(share_net_worth_bottom_50_monthly.share_net_worth_bottom_50)
log_share_net_worth_bottom_50_monthly = log_share_net_worth_bottom_50_monthly.to_frame(name='share_net_worth_bottom_50').set_index(share_net_worth_bottom_50_monthly.index)

# Calculate monthly differences
log_real_prices = np.log(real_prices)
log_real_prices_diff = log_real_prices.diff().dropna()

log_loans_monthly_diff = log_loans.loans.diff().dropna()

time = np.arange(len(log_loans_monthly_diff))  # assuming loans is a time series
X = sm.add_constant(time)  # Add constant to model for intercept

# Fit linear regression model
model = sm.OLS(log_loans_monthly_diff, X)
results = model.fit()

# Detrended data (subtract the fitted trend)
detrended_loans = log_loans_monthly_diff - results.fittedvalues


net_wealth_monthly_diff = net_wealth_monthly.net_wealth.diff().dropna()
vacancy_rate_monthly_diff = vacancy_rate_monthly.vacancy_rate.diff().dropna()
mortgage_rate_monthly_diff = mortgage_rate_monthly.mortgage_rate.diff().dropna()
active_listings_monthly_diff = active_listings.active_listings.diff().dropna()
median_house_income_monthly_diff = median_house_income_monthly.median_house_income.diff().dropna()
monthly_supply_homes_monthly_diff = monthly_supply_homes.monthly_supply_homes.diff().dropna()
labor_share_monthly_diff = labor_share_monthly.labor_share.diff().dropna()
unemployment_rate_monthly_diff = unemployment_rate.unemployment_rate.diff().dropna()
log_share_net_worth_bottom_50_monthly_diff = log_share_net_worth_bottom_50_monthly.share_net_worth_bottom_50.diff().dropna()
time = np.arange(len(log_share_net_worth_bottom_50_monthly_diff))  # assuming loans is a time series
X = sm.add_constant(time)  # Add constant to model for intercept

# Fit linear regression model
model = sm.OLS(log_share_net_worth_bottom_50_monthly_diff, X)
results = model.fit()

# Detrended data (subtract the fitted trend)
detrended_share_net_worth = log_share_net_worth_bottom_50_monthly_diff - results.fittedvalues
# FIX: Use the actual index from log_real_prices_diff
idx = log_real_prices_diff.index

# Combine all variables into a single DataFrame
data = pd.DataFrame({
    'real_prices_diff': log_real_prices_diff,
    'detrended_loans_diff': detrended_loans.reindex(idx),
    'net_wealth_diff': net_wealth_monthly_diff.reindex(idx),
    'vacancy_rate_diff': vacancy_rate_monthly_diff.reindex(idx),
    'mortgage_rate_diff': mortgage_rate_monthly_diff.reindex(idx),
    # 'active_listings_diff': active_listings_monthly_diff.reindex(idx),
    'median_house_income_diff': median_house_income_monthly_diff.reindex(idx),
    'monthly_supply_homes_diff': monthly_supply_homes_monthly_diff.reindex(idx),
    'labor_share_diff': labor_share_monthly_diff.reindex(idx),
    'unemployment_rate_diff': unemployment_rate_monthly_diff.reindex(idx),
    'detrended_share_net_worth': detrended_share_net_worth.reindex(idx)
}, index=idx)

# Check for missing values
print("\nMissing values per column:")
print(data.isnull().sum())
print(f"\nData shape: {data.shape}")
print(f"\nDate range: {data.index.min()} to {data.index.max()}")
# remove rows with any missing values
data = data.dropna()
print(f"\nData shape after dropping missing values: {data.shape}")

observation_date
1963-01-01    30.44
1963-02-01    30.48
1963-03-01    30.51
1963-04-01    30.48
1963-05-01    30.51
Name: cpi, dtype: float64
Rows after filtering: 752


ValueError: Data must be 1-dimensional, got ndarray of shape (751, 1) instead

In [ ]:
from statsmodels.tsa.api import VAR
import numpy as np


# Fit VAR model and test different lag orders
model = VAR(data)
lag_order_results = model.select_order(maxlags=12)
print(lag_order_results.summary())

# # This will show AIC, BIC, HQIC for different lags
# # Lower values = better
# optimal_lag = lag_order_results.aic  # or .bic, .hqic

In [ ]:
from statsmodels.tsa.stattools import ccf
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

# Check cross-correlations between each predictor and target
predictors = [col for col in data.columns if col != 'real_prices_diff']

# test stationarity for each predictor
for predictor in predictors:
    clean_data = data[['real_prices_diff', predictor]].dropna()
    adf_result = adfuller(clean_data[predictor])
    print(f"ADF Statistic for {predictor}: {adf_result[0]}")
    print(f"p-value: {adf_result[1]}")
    for key, value in adf_result[4].items():
        print('Critical Value (%s): %.3f' % (key, value))
    print("\n")


In [ ]:


for predictor in predictors:
    clean_data = data[['real_prices_diff', predictor]].dropna()
    correlations = ccf(clean_data[predictor], clean_data['real_prices_diff'])[:13]
    
    plt.figure(figsize=(10, 3))
    plt.stem(range(len(correlations)), correlations)
    plt.title(f'Cross-correlation: {predictor} -> real_prices_diff')
    plt.xlabel('Lag')
    plt.ylabel('Correlation')
    plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    plt.show()
    
    # Print significant lags (rule of thumb: |corr| > 2/sqrt(n))
    threshold = 2/np.sqrt(len(clean_data))
    significant_lags = [i for i, corr in enumerate(correlations) if abs(corr) > threshold]
    print(f"{predictor}: Significant lags at {significant_lags}\n")

In [ ]:
# Granger causality tests
from statsmodels.tsa.stattools import grangercausalitytests

# Test if lagged values of each predictor help predict target
for predictor in predictors:
    test_data = data[['real_prices_diff', predictor]].dropna()
    print(f"\n--- Granger Causality: {predictor} -> real_prices_diff ---")
    try:
        results = grangercausalitytests(test_data, maxlag=12, verbose=True)
    except:
        print("Test failed (possibly due to insufficient data)")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

# Create lagged features
def create_lagged_features(df, target_col, max_lag=12):
    df_lagged = df.copy()
    predictors = [col for col in df.columns if col != target_col]
    
    for col in predictors:
        for lag in range(1, max_lag + 1):
            df_lagged[f'{col}_lag{lag}'] = df[col].shift(lag)
    
    return df_lagged.dropna()

# Create features
df_with_lags = create_lagged_features(data, 'real_prices_diff', max_lag=12)

X = df_with_lags.drop('real_prices_diff', axis=1)
y = df_with_lags['real_prices_diff']

# Use Random Forest feature importance
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Plot feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(20))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import ccf, grangercausalitytests
from scipy import stats
import seaborn as sns

# Assuming your data DataFrame is already defined
# data = pd.DataFrame({...})

def find_significant_lags(data, target_col='real_prices_diff', max_lag=12):
    """
    Find significant lags using practical methods that work well with differenced data
    Focus on: Cross-correlation, Individual regressions, and PACF
    """
    predictors = [col for col in data.columns if col != target_col]
    results = {}
    
    print("="*80)
    print("FINDING SIGNIFICANT LAGS FOR PREDICTORS")
    print("="*80)
    
    message = ""
    
    for predictor in predictors:
        print(f"\n{'='*80}")
        print(f"Analyzing: {predictor}")
        print(f"{'='*80}")
        
        # Clean data
        clean_data = data[[target_col, predictor]].dropna()
        n = len(clean_data)
        
        if n < max_lag + 10:
            print(f"⚠️  Insufficient data (n={n}). Skipping.")
            continue
        
        # Method 1: Cross-Correlation Function (Most reliable for differenced data)
        print("\n--- Cross-Correlation Analysis ---")
        ccf_values = ccf(clean_data[predictor], clean_data[target_col], adjusted=False)[:max_lag+1]
        
        # Significance threshold: ±1.96/sqrt(n) for 95% confidence
        threshold = 1.96 / np.sqrt(n)
        
        significant_lags_ccf = []
        for lag in range(max_lag + 1):
            if abs(ccf_values[lag]) > threshold:
                significant_lags_ccf.append(lag)
                sig_marker = '***' if abs(ccf_values[lag]) > 2*threshold else '**'
                print(f"  Lag {lag}: {ccf_values[lag]:.4f} {sig_marker}")
        
        if not significant_lags_ccf:
            print("  No significant lags found")
        
        # Method 2: Individual Lag Regressions (More powerful than Granger for single predictors)
        print("\n--- Individual Lag Regression t-tests ---")
        from statsmodels.api import OLS, add_constant
        
        significant_lags_regression = []
        regression_results = {}
        
        for lag in range(1, max_lag + 1):
            # Create lagged predictor
            lagged_data = pd.DataFrame({
                'y': clean_data[target_col].values[lag:],
                'x_lag': clean_data[predictor].values[:-lag]
            })
            
            X = add_constant(lagged_data['x_lag'])
            y = lagged_data['y']
            
            model = OLS(y, X).fit()
            p_value = model.pvalues['x_lag']
            coef = model.params['x_lag']
            t_stat = model.tvalues['x_lag']
            regression_results[lag] = {'coef': coef, 'p_value': p_value, 't_stat': t_stat}
            
            if p_value < 0.1:  # Using 10% threshold for screening
                significant_lags_regression.append(lag)
                sig_marker = '***' if p_value < 0.01 else '**' if p_value < 0.05 else '*'
                print(f"  Lag {lag}: coef = {coef:.6f}, t-stat = {t_stat:.3f}, p = {p_value:.4f} {sig_marker}")
                message += f"Predictor {predictor} has significant lag {lag} (p={p_value:.4f})\n"
        
        if not significant_lags_regression:
            print("  No significant lags found (p < 0.1)")
        
        # Method 3: Stepwise regression (test multiple lags together)
        print("\n--- Stepwise Regression (controlling for other lags) ---")
        from statsmodels.api import OLS, add_constant
        
        # Test all lags together
        lag_data = pd.DataFrame({'y': clean_data[target_col].values[max_lag:]})
        
        for lag in range(1, max_lag + 1):
            lag_data[f'x_lag{lag}'] = clean_data[predictor].values[max_lag-lag:-lag]
        
        X_full = add_constant(lag_data.drop('y', axis=1))
        y_full = lag_data['y']
        
        model_full = OLS(y_full, X_full).fit()
        
        print(f"\n  Full model R² = {model_full.rsquared:.4f}, Adj. R² = {model_full.rsquared_adj:.4f}")
        print("  Significant lags in joint model:")
        
        joint_significant = []
        for lag in range(1, max_lag + 1):
            col_name = f'x_lag{lag}'
            if col_name in model_full.pvalues.index:
                p_val = model_full.pvalues[col_name]
                if p_val < 0.1:
                    joint_significant.append(lag)
                    coef = model_full.params[col_name]
                    sig_marker = '***' if p_val < 0.01 else '**' if p_val < 0.05 else '*'
                    print(f"    Lag {lag}: coef = {coef:.6f}, p = {p_val:.4f} {sig_marker}")
                    message += f"Predictor {predictor} has significant lag {lag} (p={p_val:.4f})\n"
        
        if not joint_significant:
            print("    No significant lags in joint model")
        
        # Store results
        results[predictor] = {
            'ccf_values': ccf_values,
            'ccf_threshold': threshold,
            'significant_lags_ccf': significant_lags_ccf,
            'significant_lags_regression': significant_lags_regression,
            'regression_results': regression_results,
            'joint_significant': joint_significant,
            'full_model_r2': model_full.rsquared_adj,
            'n_obs': n
        }
        
        # Summary recommendation
        print("\n--- RECOMMENDATION ---")
        # Combine evidence from different methods
        all_significant = set(significant_lags_ccf) | set(significant_lags_regression) | set(joint_significant)
        
        if all_significant:
            recommended_lags = sorted(list(all_significant))
            print(f"  ✓ Recommended lags to include: {recommended_lags}")
            message += f"Predictor {predictor} has significant lags: {recommended_lags}\n"
            
            # Categorize by strength of evidence
            strong_lags = set(significant_lags_ccf) & set(significant_lags_regression)
            if strong_lags:
                print(f"  ✓✓ Strongest evidence for lags: {sorted(list(strong_lags))}")
                message += f"Strong evidence for lags: {sorted(list(strong_lags))}\n"
        else:
            print(f"  ✗ No significant lags found. Consider excluding this predictor or using contemporaneous value only.")
    
    print(message)
    return results


def plot_lag_analysis(results, data, target_col='real_prices_diff'):
    """
    Create visualizations of lag relationships
    """
    predictors = list(results.keys())
    n_predictors = len(predictors)
    
    fig, axes = plt.subplots(n_predictors, 2, figsize=(14, 4*n_predictors))
    if n_predictors == 1:
        axes = axes.reshape(1, -1)
    
    for idx, predictor in enumerate(predictors):
        res = results[predictor]
        max_lag = len(res['ccf_values']) - 1
        lags = range(max_lag + 1)
        
        # Plot 1: Cross-correlation
        ax1 = axes[idx, 0]
        ax1.stem(lags, res['ccf_values'], basefmt=' ')
        ax1.axhline(y=res['ccf_threshold'], color='r', linestyle='--', label='95% CI')
        ax1.axhline(y=-res['ccf_threshold'], color='r', linestyle='--')
        ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
        ax1.set_xlabel('Lag')
        ax1.set_ylabel('Cross-correlation')
        ax1.set_title(f'{predictor}\nCross-correlation with {target_col}')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Highlight significant lags
        for lag in res['significant_lags_ccf']:
            ax1.axvspan(lag-0.3, lag+0.3, alpha=0.2, color='green')
        
        # Plot 2: Regression coefficients and p-values
        ax2 = axes[idx, 1]
        reg_results = res['regression_results']
        lags_reg = list(reg_results.keys())
        coefs = [reg_results[lag]['coef'] for lag in lags_reg]
        p_values = [reg_results[lag]['p_value'] for lag in lags_reg]
        
        # Bar plot of coefficients with color based on significance
        colors = ['green' if p < 0.05 else 'orange' if p < 0.1 else 'gray' 
                  for p in p_values]
        ax2.bar(lags_reg, coefs, color=colors, alpha=0.6)
        ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
        ax2.set_xlabel('Lag')
        ax2.set_ylabel('Regression Coefficient')
        ax2.set_title(f'{predictor}\nIndividual Lag Coefficients\n(Green: p<0.05, Orange: p<0.1)')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def create_optimal_lag_model(data, results, target_col='real_prices_diff', 
                             min_lag_evidence=1):
    """
    Create a regression model using the identified significant lags
    
    Parameters:
    -----------
    min_lag_evidence : int
        Minimum number of methods that must agree on lag significance (1-3)
    """
    from statsmodels.api import OLS, add_constant
    
    print("\n" + "="*80)
    print("BUILDING OPTIMAL LAG MODEL")
    print("="*80)
    
    # Collect all recommended lags
    model_features = {}
    
    for predictor, res in results.items():
        # Count evidence for each lag
        all_lags = set(res['significant_lags_ccf']) | \
                   set(res['significant_lags_regression']) | \
                   set(res.get('joint_significant', []))
        
        if all_lags:
            # For each lag, count how many methods found it significant
            lag_evidence = {}
            for lag in all_lags:
                evidence = 0
                if lag in res['significant_lags_ccf']:
                    evidence += 1
                if lag in res['significant_lags_regression']:
                    evidence += 1
                if lag in res.get('joint_significant', []):
                    evidence += 1
                lag_evidence[lag] = evidence
            
            # Keep lags with sufficient evidence
            selected_lags = [lag for lag, ev in lag_evidence.items() 
                           if ev >= min_lag_evidence]
            
            if selected_lags:
                model_features[predictor] = sorted(selected_lags)
                print(f"\n{predictor}:")
                print(f"  Selected lags: {selected_lags}")
    
    # Build the model
    print("\n" + "-"*80)
    print("Building regression model...")
    
    # Determine maximum lag needed
    max_lag_needed = max([max(lags) for lags in model_features.values()])
    
    # Create lagged features
    model_data = pd.DataFrame({target_col: data[target_col].values[max_lag_needed:]})
    
    for predictor, lags in model_features.items():
        for lag in lags:
            col_name = f'{predictor}_lag{lag}'
            model_data[col_name] = data[predictor].values[max_lag_needed-lag:-lag]
    
    # Remove NaN values
    model_data = model_data.dropna()
    
    print(f"\nModel data shape: {model_data.shape}")
    print(f"Number of features: {model_data.shape[1] - 1}")
    print(f"Number of observations: {model_data.shape[0]}")
    
    # Fit model
    y = model_data[target_col]
    X = add_constant(model_data.drop(target_col, axis=1))
    
    model = OLS(y, X).fit()
    
    print("\n" + "="*80)
    print("MODEL RESULTS")
    print("="*80)
    print(model.summary())
    
    return model, model_data, model_features


# Example usage:
results = find_significant_lags(data, max_lag=12)
plot_lag_analysis(results, data)
model, model_data, features = create_optimal_lag_model(data, results, min_lag_evidence=1)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import ccf, grangercausalitytests
from scipy import stats
import seaborn as sns

# Assuming your data DataFrame is already defined
# data = pd.DataFrame({...})

def find_significant_lags(data, target_col='real_prices_diff', max_lag=12):
    """
    Find significant lags using practical methods that work well with differenced data
    Focus on: Cross-correlation, Individual regressions, and PACF
    """
    predictors = [col for col in data.columns if col != target_col]
    results = {}
    
    print("="*80)
    print("FINDING SIGNIFICANT LAGS FOR PREDICTORS")
    print("="*80)
    
    for predictor in predictors:
        print(f"\n{'='*80}")
        print(f"Analyzing: {predictor}")
        print(f"{'='*80}")
        
        # Clean data
        clean_data = data[[target_col, predictor]].dropna()
        n = len(clean_data)
        
        if n < max_lag + 10:
            print(f"⚠️  Insufficient data (n={n}). Skipping.")
            continue
        
        # Method 1: Cross-Correlation Function (Most reliable for differenced data)
        print("\n--- Cross-Correlation Analysis ---")
        ccf_values = ccf(clean_data[predictor], clean_data[target_col], adjusted=False)[:max_lag+1]
        
        # Significance threshold: ±1.96/sqrt(n) for 95% confidence
        threshold = 1.96 / np.sqrt(n)
        
        significant_lags_ccf = []
        for lag in range(max_lag + 1):
            if abs(ccf_values[lag]) > threshold:
                significant_lags_ccf.append(lag)
                sig_marker = '***' if abs(ccf_values[lag]) > 2*threshold else '**'
                print(f"  Lag {lag}: {ccf_values[lag]:.4f} {sig_marker}")
        
        if not significant_lags_ccf:
            print("  No significant lags found")
        
        # Method 2: Individual Lag Regressions (More powerful than Granger for single predictors)
        print("\n--- Individual Lag Regression t-tests ---")
        from statsmodels.api import OLS, add_constant
        
        significant_lags_regression = []
        regression_results = {}
        
        for lag in range(1, max_lag + 1):
            # Create lagged predictor
            lagged_data = pd.DataFrame({
                'y': clean_data[target_col].values[lag:],
                'x_lag': clean_data[predictor].values[:-lag]
            })
            
            X = add_constant(lagged_data['x_lag'])
            y = lagged_data['y']
            
            model = OLS(y, X).fit()
            p_value = model.pvalues['x_lag']
            coef = model.params['x_lag']
            t_stat = model.tvalues['x_lag']
            regression_results[lag] = {'coef': coef, 'p_value': p_value, 't_stat': t_stat}
            
            if p_value < 0.1:  # Using 10% threshold for screening
                significant_lags_regression.append(lag)
                sig_marker = '***' if p_value < 0.01 else '**' if p_value < 0.05 else '*'
                print(f"  Lag {lag}: coef = {coef:.6f}, t-stat = {t_stat:.3f}, p = {p_value:.4f} {sig_marker}")
        
        if not significant_lags_regression:
            print("  No significant lags found (p < 0.1)")
        
        # Method 3: Stepwise regression (test multiple lags together)
        print("\n--- Stepwise Regression (controlling for other lags) ---")
        from statsmodels.api import OLS, add_constant
        
        # Test all lags together
        lag_data = pd.DataFrame({'y': clean_data[target_col].values[max_lag:]})
        
        for lag in range(1, max_lag + 1):
            lag_data[f'x_lag{lag}'] = clean_data[predictor].values[max_lag-lag:-lag]
        
        X_full = add_constant(lag_data.drop('y', axis=1))
        y_full = lag_data['y']
        
        model_full = OLS(y_full, X_full).fit()
        
        print(f"\n  Full model R² = {model_full.rsquared:.4f}, Adj. R² = {model_full.rsquared_adj:.4f}")
        print("  Significant lags in joint model:")
        
        joint_significant = []
        for lag in range(1, max_lag + 1):
            col_name = f'x_lag{lag}'
            if col_name in model_full.pvalues.index:
                p_val = model_full.pvalues[col_name]
                if p_val < 0.1:
                    joint_significant.append(lag)
                    coef = model_full.params[col_name]
                    sig_marker = '***' if p_val < 0.01 else '**' if p_val < 0.05 else '*'
                    print(f"    Lag {lag}: coef = {coef:.6f}, p = {p_val:.4f} {sig_marker}")
        
        if not joint_significant:
            print("    No significant lags in joint model")
        
        # Store results
        results[predictor] = {
            'ccf_values': ccf_values,
            'ccf_threshold': threshold,
            'significant_lags_ccf': significant_lags_ccf,
            'significant_lags_regression': significant_lags_regression,
            'regression_results': regression_results,
            'joint_significant': joint_significant,
            'full_model_r2': model_full.rsquared_adj,
            'n_obs': n
        }
        
        # Summary recommendation
        print("\n--- RECOMMENDATION ---")
        # Combine evidence from different methods
        all_significant = set(significant_lags_ccf) | set(significant_lags_regression) | set(joint_significant)
        
        if all_significant:
            recommended_lags = sorted(list(all_significant))
            print(f"  ✓ Recommended lags to include: {recommended_lags}")
            
            # Categorize by strength of evidence
            strong_lags = set(significant_lags_ccf) & set(significant_lags_regression)
            if strong_lags:
                print(f"  ✓✓ Strongest evidence for lags: {sorted(list(strong_lags))}")
        else:
            print(f"  ✗ No significant lags found. Consider excluding this predictor or using contemporaneous value only.")
    
    return results


def plot_lag_analysis(results, data, target_col='real_prices_diff'):
    """
    Create visualizations of lag relationships
    """
    predictors = list(results.keys())
    n_predictors = len(predictors)
    
    fig, axes = plt.subplots(n_predictors, 2, figsize=(14, 4*n_predictors))
    if n_predictors == 1:
        axes = axes.reshape(1, -1)
    
    for idx, predictor in enumerate(predictors):
        res = results[predictor]
        max_lag = len(res['ccf_values']) - 1
        lags = range(max_lag + 1)
        
        # Plot 1: Cross-correlation
        ax1 = axes[idx, 0]
        ax1.stem(lags, res['ccf_values'], basefmt=' ')
        ax1.axhline(y=res['ccf_threshold'], color='r', linestyle='--', label='95% CI')
        ax1.axhline(y=-res['ccf_threshold'], color='r', linestyle='--')
        ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
        ax1.set_xlabel('Lag')
        ax1.set_ylabel('Cross-correlation')
        ax1.set_title(f'{predictor}\nCross-correlation with {target_col}')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Highlight significant lags
        for lag in res['significant_lags_ccf']:
            ax1.axvspan(lag-0.3, lag+0.3, alpha=0.2, color='green')
        
        # Plot 2: Regression coefficients and p-values
        ax2 = axes[idx, 1]
        reg_results = res['regression_results']
        lags_reg = list(reg_results.keys())
        coefs = [reg_results[lag]['coef'] for lag in lags_reg]
        p_values = [reg_results[lag]['p_value'] for lag in lags_reg]
        
        # Bar plot of coefficients with color based on significance
        colors = ['green' if p < 0.05 else 'orange' if p < 0.1 else 'gray' 
                  for p in p_values]
        ax2.bar(lags_reg, coefs, color=colors, alpha=0.6)
        ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
        ax2.set_xlabel('Lag')
        ax2.set_ylabel('Regression Coefficient')
        ax2.set_title(f'{predictor}\nIndividual Lag Coefficients\n(Green: p<0.05, Orange: p<0.1)')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def create_optimal_lag_model(data, results, target_col='real_prices_diff', 
                             min_lag_evidence=1):
    """
    Create a regression model using the identified significant lags
    
    Parameters:
    -----------
    min_lag_evidence : int
        Minimum number of methods that must agree on lag significance (1-3)
    """
    from statsmodels.api import OLS, add_constant
    
    print("\n" + "="*80)
    print("BUILDING OPTIMAL LAG MODEL")
    print("="*80)
    
    # Collect all recommended lags
    model_features = {}
    
    for predictor, res in results.items():
        # Count evidence for each lag
        all_lags = set(res['significant_lags_ccf']) | \
                   set(res['significant_lags_regression']) | \
                   set(res.get('joint_significant', []))
        
        if all_lags:
            # For each lag, count how many methods found it significant
            lag_evidence = {}
            for lag in all_lags:
                evidence = 0
                if lag in res['significant_lags_ccf']:
                    evidence += 1
                if lag in res['significant_lags_regression']:
                    evidence += 1
                if lag in res.get('joint_significant', []):
                    evidence += 1
                lag_evidence[lag] = evidence
            
            # Keep lags with sufficient evidence
            selected_lags = [lag for lag, ev in lag_evidence.items() 
                           if ev >= min_lag_evidence]
            
            if selected_lags:
                model_features[predictor] = sorted(selected_lags)
                print(f"\n{predictor}:")
                print(f"  Selected lags: {selected_lags}")
    
    # Build the model
    print("\n" + "-"*80)
    print("Building regression model...")
    
    # Determine maximum lag needed
    max_lag_needed = max([max(lags) for lags in model_features.values()])
    
    # Create lagged features
    model_data = pd.DataFrame({target_col: data[target_col].values[max_lag_needed:]})
    
    for predictor, lags in model_features.items():
        for lag in lags:
            col_name = f'{predictor}_lag{lag}'
            model_data[col_name] = data[predictor].values[max_lag_needed-lag:-lag]
    
    # Remove NaN values
    model_data = model_data.dropna()
    
    print(f"\nModel data shape: {model_data.shape}")
    print(f"Number of features: {model_data.shape[1] - 1}")
    print(f"Number of observations: {model_data.shape[0]}")
    
    # Fit model
    y = model_data[target_col]
    X = add_constant(model_data.drop(target_col, axis=1))
    
    model = OLS(y, X).fit()
    
    print("\n" + "="*80)
    print("MODEL RESULTS")
    print("="*80)
    print(model.summary())
    
    return model, model_data, model_features


def forecast_one_step(model, data, model_features, target_col='real_prices_diff'):
    """
    Make a one-step ahead forecast using the fitted model
    
    Parameters:
    -----------
    model : statsmodels regression results
        The fitted OLS model
    data : pd.DataFrame
        The original data with all variables
    model_features : dict
        Dictionary with predictor names as keys and lists of lags as values
    target_col : str
        Name of the target variable
        
    Returns:
    --------
    forecast : float
        One-step ahead forecast
    forecast_df : pd.DataFrame
        DataFrame with the feature values used for prediction
    """
    from statsmodels.api import add_constant
    
    # Get the most recent values needed for prediction
    max_lag_needed = max([max(lags) for lags in model_features.values()])
    
    # Create the feature vector for prediction
    forecast_features = {}
    
    print("\n" + "="*80)
    print("ONE-STEP AHEAD FORECAST")
    print("="*80)
    print(f"\nUsing most recent {max_lag_needed} observations for lagged features:")
    print("-"*80)
    
    for predictor, lags in model_features.items():
        print(f"\n{predictor}:")
        for lag in lags:
            # Get the value from 'lag' periods ago
            feature_name = f'{predictor}_lag{lag}'
            feature_value = data[predictor].iloc[-lag]
            forecast_features[feature_name] = feature_value
            
            # Show which historical value is being used
            date_used = data.index[-lag] if hasattr(data.index[-lag], 'strftime') else f"observation t-{lag}"
            print(f"  {feature_name}: {feature_value:.6f} (from {date_used})")
    
    # Create DataFrame for prediction
    forecast_df = pd.DataFrame([forecast_features])
    
    # Add constant term
    forecast_df_with_const = add_constant(forecast_df, has_constant='add')
    
    # Make sure columns match the model
    model_cols = [col for col in model.model.exog_names if col in forecast_df_with_const.columns or col == 'const']
    forecast_df_with_const = forecast_df_with_const[model_cols]
    
    # Generate forecast
    forecast = model.predict(forecast_df_with_const)[0]
    
    # Get prediction interval
    prediction = model.get_prediction(forecast_df_with_const)
    pred_summary = prediction.summary_frame(alpha=0.05)
    
    print("\n" + "="*80)
    print("FORECAST RESULTS")
    print("="*80)
    print(f"\nOne-step ahead forecast: {forecast:.6f}")
    print(f"95% Prediction Interval: [{pred_summary['obs_ci_lower'].values[0]:.6f}, {pred_summary['obs_ci_upper'].values[0]:.6f}]")
    print(f"Standard Error: {pred_summary['mean_se'].values[0]:.6f}")
    
    return forecast, forecast_df, pred_summary


def rolling_forecast_validation(data, model_features, target_col='real_prices_diff', 
                                test_size=0.2, expanding_window=True):
    """
    Perform rolling one-step ahead forecasts for model validation
    
    Parameters:
    -----------
    data : pd.DataFrame
        The full dataset
    model_features : dict
        Dictionary with predictor names as keys and lists of lags as values
    target_col : str
        Name of the target variable
    test_size : float
        Proportion of data to use for testing (default 0.2 = 20%)
    expanding_window : bool
        If True, use expanding window (training set grows each iteration)
        If False, use rolling window (training set size stays constant)
        
    Returns:
    --------
    forecast_results : pd.DataFrame
        DataFrame with actual values, forecasts, and errors
    """
    from statsmodels.api import OLS, add_constant
    
    print("\n" + "="*80)
    print("ROLLING ONE-STEP AHEAD FORECAST VALIDATION")
    print("="*80)
    
    max_lag_needed = max([max(lags) for lags in model_features.values()])
    
    # Initialize results storage
    forecasts = []
    actuals = []
    dates = []
    
    # Calculate train/test split
    total_obs = len(data)
    n_forecast = int(total_obs * test_size)
    min_train_size = total_obs - n_forecast
    
    window_type = "expanding" if expanding_window else "rolling"
    
    print(f"\nTotal observations: {total_obs}")
    print(f"Training set (initial): {min_train_size} observations ({(1-test_size)*100:.0f}%)")
    print(f"Test set: {n_forecast} observations ({test_size*100:.0f}%)")
    print(f"Window type: {window_type}")
    print(f"Max lag used: {max_lag_needed}")
    print("-"*80)
    
    print(f"\nGenerating {n_forecast} one-step ahead forecasts...")
    print(f"Window type: {window_type}")
    
    for i in range(n_forecast):
        if expanding_window:
            # Expanding window: training set grows
            train_start = 0
            train_end = min_train_size + i
        else:
            # Rolling window: training set size stays constant
            train_start = i
            train_end = min_train_size + i
        
        # Create training data
        train_data = data.iloc[train_start:train_end]
        
        # Build model on training data
        model_train_data = pd.DataFrame({target_col: train_data[target_col].values[max_lag_needed:]})
        
        for predictor, lags in model_features.items():
            for lag in lags:
                col_name = f'{predictor}_lag{lag}'
                model_train_data[col_name] = train_data[predictor].values[max_lag_needed-lag:-lag]
        
        model_train_data = model_train_data.dropna()
        
        # Fit model
        y_train = model_train_data[target_col]
        X_train = add_constant(model_train_data.drop(target_col, axis=1))
        model = OLS(y_train, X_train).fit()
        
        # Make one-step ahead forecast
        forecast_features = {}
        for predictor, lags in model_features.items():
            for lag in lags:
                feature_name = f'{predictor}_lag{lag}'
                forecast_features[feature_name] = train_data[predictor].iloc[-lag]
        
        forecast_df = pd.DataFrame([forecast_features])
        forecast_df_with_const = add_constant(forecast_df, has_constant='add')
        
        # Ensure column order matches
        forecast_df_with_const = forecast_df_with_const[model.model.exog_names]
        
        # Generate forecast
        forecast = model.predict(forecast_df_with_const)[0]
        
        # Get actual value
        actual = data[target_col].iloc[train_end]
        
        forecasts.append(forecast)
        actuals.append(actual)
        dates.append(data.index[train_end])
        
        if (i + 1) % max(1, n_forecast // 10) == 0 or i == 0 or i == n_forecast - 1:
            print(f"Forecast {i+1}/{n_forecast}: Train size={len(y_train)}, Predicted={forecast:.6f}, Actual={actual:.6f}, Error={actual-forecast:.6f}")
    
    # Create results DataFrame
    forecast_results = pd.DataFrame({
        'date': dates,
        'actual': actuals,
        'forecast': forecasts,
        'error': np.array(actuals) - np.array(forecasts)
    })
    
    forecast_results['abs_error'] = forecast_results['error'].abs()
    forecast_results['squared_error'] = forecast_results['error'] ** 2
    forecast_results['abs_pct_error'] = (forecast_results['abs_error'] / forecast_results['actual'].abs()) * 100
    
    # Calculate error metrics
    print("\n" + "="*80)
    print("FORECAST ACCURACY METRICS")
    print("="*80)
    print(f"\nMean Error (ME): {forecast_results['error'].mean():.6f}")
    print(f"Mean Absolute Error (MAE): {forecast_results['abs_error'].mean():.6f}")
    print(f"Root Mean Squared Error (RMSE): {np.sqrt(forecast_results['squared_error'].mean()):.6f}")
    print(f"Mean Absolute Percentage Error (MAPE): {forecast_results['abs_pct_error'].mean():.2f}%")
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(forecast_results['date'], forecast_results['actual'], 'o-', label='Actual', linewidth=2)
    plt.plot(forecast_results['date'], forecast_results['forecast'], 's--', label='Forecast', linewidth=2)
    plt.xlabel('Date')
    plt.ylabel(target_col)
    plt.title('Rolling One-Step Ahead Forecasts vs Actual Values')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Plot forecast errors
    plt.figure(figsize=(12, 4))
    plt.bar(range(len(forecast_results)), forecast_results['error'], alpha=0.6)
    plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    plt.xlabel('Forecast Number')
    plt.ylabel('Forecast Error')
    plt.title('One-Step Ahead Forecast Errors')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return forecast_results


# Example usage:
# 1. Find significant lags
results = find_significant_lags(data, max_lag=12)
plot_lag_analysis(results, data)

# 2. Build model on full data (for exploration)
model, model_data, features = create_optimal_lag_model(data, results, min_lag_evidence=1)

# 3. Make a single one-step ahead forecast
forecast, forecast_df, pred_summary = forecast_one_step(model, data, features)

# 4. Validate with rolling forecasts on last 20% of data
forecast_results = rolling_forecast_validation(data, features, test_size=0.2, expanding_window=True)

# Alternative: Use rolling window instead of expanding
#forecast_results = rolling_forecast_validation(data, features, test_size=0.2, expanding_window=False)

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Apply second differencing
loans_diff_2 = data['loans_diff'].diff().dropna()
share_net_worth_bottom_50_diff_2 = data['share_net_worth_bottom_50_diff'].diff().dropna()

# ADF Test for second differenced series
adf_result_loans_2 = adfuller(loans_diff_2)
adf_result_share_net_worth_2 = adfuller(share_net_worth_bottom_50_diff_2)

print(f"ADF Statistic for loans_diff_2: {adf_result_loans_2[0]}")
print(f"p-value for loans_diff_2: {adf_result_loans_2[1]}")

print(f"ADF Statistic for share_net_worth_bottom_50_diff_2: {adf_result_share_net_worth_2[0]}")
print(f"p-value for share_net_worth_bottom_50_diff_2: {adf_result_share_net_worth_2[1]}")


In [ ]:
##### PCA and Regression Analysis
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

X = data[[
'loans_diff',
'net_wealth_diff',
'vacancy_rate_diff',
'mortgage_rate_diff',
# 'active_listings_diff',
'median_house_income_diff',
'monthly_supply_homes_diff',
'labor_share_diff',
'unemployment_rate_diff',
'share_net_worth_bottom_50_diff' ]]
y = data['real_prices_diff']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print("\n" + "=" * 60)
print("STEP 2: STANDARDIZATION")
print("=" * 60)
# Standardize features (required for PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeatures standardized (mean=0, std=1)")
print("Training set shape:", X_train_scaled.shape)
print("Test set shape:", X_test_scaled.shape)

print("\n" + "=" * 60)
print("STEP 3: PCA FACTOR ANALYSIS")
print("=" * 60)

# Perform PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Display explained variance
print("\nExplained variance by each component:")
for i, var in enumerate(pca.explained_variance_ratio_, 1):
    print(f"  PC{i}: {var:.4f} ({var*100:.2f}%)")

print(f"\nCumulative explained variance:")
cumsum = np.cumsum(pca.explained_variance_ratio_)
for i, var in enumerate(cumsum, 1):
    print(f"  PC1-PC{i}: {var:.4f} ({var*100:.2f}%)")

# Component loadings
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i}' for i in range(1, len(pca.components_) + 1)],
    index=X.columns
)
print("\nPCA Loadings (Component Matrix):")
print(loadings.round(3))

print("\n" + "=" * 60)
print("STEP 4: DETERMINE NUMBER OF FACTORS")
print("=" * 60)

# Kaiser criterion: eigenvalues > 1
eigenvalues = pca.explained_variance_
print("\nEigenvalues:")
for i, ev in enumerate(eigenvalues, 1):
    print(f"  PC{i}: {ev:.4f}", "✓ (Keep)" if ev > 1 else "  (Drop)")

n_components_kaiser = sum(eigenvalues > 1)
print(f"\nKaiser criterion suggests keeping {n_components_kaiser} component(s)")

# Choose based on variance threshold (e.g., 80%)
variance_threshold = 0.80
n_components_var = np.argmax(cumsum >= variance_threshold) + 1
print(f"To explain {variance_threshold*100}% variance: {n_components_var} component(s)")

print("\n" + "=" * 60)
print("STEP 5: REGRESSION WITH PRINCIPAL COMPONENTS")
print("=" * 60)

# Use the number of components that explain ~80-90% variance
n_final = max(n_components_kaiser, n_components_var)
print(f"\nUsing {n_final} principal component(s) for regression")

X_train_pca_reduced = X_train_pca[:, :n_final]
X_test_pca_reduced = X_test_pca[:, :n_final]

# Fit regression model
model = LinearRegression()
model.fit(X_train_pca_reduced, y_train)

# Predictions
y_train_pred = model.predict(X_train_pca_reduced)
y_test_pred = model.predict(X_test_pca_reduced)

# Evaluate
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\nModel Performance:")
print(f"  Training R²: {train_r2:.4f}")
print(f"  Test R²: {test_r2:.4f}")
print(f"  Training RMSE: {train_rmse:.6f}")
print(f"  Test RMSE: {test_rmse:.6f}")
print(f"  Training MAE: {train_mae:.6f}")
print(f"  Test MAE: {test_mae:.6f}")

print("\nRegression Coefficients (for PCs):")
for i, coef in enumerate(model.coef_, 1):
    print(f"  PC{i}: {coef:.4f}")
print(f"  Intercept: {model.intercept_:.6f}")

# Interpretation: Map PC coefficients back to original variables
print("\n" + "=" * 60)
print("INTERPRETATION: Effect on Price Changes")
print("=" * 60)
print("\nApproximate contribution of original variables:")
print("(PC coefficients × loadings)")

original_effects = np.dot(loadings.iloc[:, :n_final], model.coef_)
for var, effect in zip(X.columns, original_effects):
    print(f"  {var}: {effect:.4f}")

print("\n" + "=" * 60)
print("STEP 6: VISUALIZATION")
print("=" * 60)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Scree plot
axes[0, 0].bar(range(1, len(eigenvalues) + 1), eigenvalues, alpha=0.7, color='steelblue')
axes[0, 0].axhline(y=1, color='r', linestyle='--', label='Kaiser criterion', linewidth=2)
axes[0, 0].set_xlabel('Principal Component', fontsize=11)
axes[0, 0].set_ylabel('Eigenvalue', fontsize=11)
axes[0, 0].set_title('Scree Plot', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Cumulative variance explained
axes[0, 1].plot(range(1, len(cumsum) + 1), cumsum, marker='o', linewidth=2, 
                color='steelblue', markersize=8)
axes[0, 1].axhline(y=0.8, color='r', linestyle='--', label='80% variance', linewidth=2)
axes[0, 1].set_xlabel('Number of Components', fontsize=11)
axes[0, 1].set_ylabel('Cumulative Explained Variance', fontsize=11)
axes[0, 1].set_title('Cumulative Variance Explained', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim([0, 1.05])

# 3. Component loadings heatmap
sns.heatmap(loadings.iloc[:, :n_final], annot=True, cmap='RdBu_r', center=0, 
            ax=axes[1, 0], cbar_kws={'label': 'Loading'}, fmt='.3f',
            vmin=-1, vmax=1, linewidths=0.5)
axes[1, 0].set_title('PCA Loadings Heatmap', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Principal Component', fontsize=11)
axes[1, 0].set_ylabel('Original Variable (Differenced)', fontsize=11)

# 4. Actual vs Predicted
axes[1, 1].scatter(y_test, y_test_pred, alpha=0.6, color='steelblue', 
                   edgecolors='black', s=50)
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, 
                label='Perfect prediction')
axes[1, 1].set_xlabel('Actual Price Change (Log Diff)', fontsize=11)
axes[1, 1].set_ylabel('Predicted Price Change (Log Diff)', fontsize=11)
axes[1, 1].set_title(f'Actual vs Predicted (Test Set)\nR² = {test_r2:.4f}', 
                     fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pca_analysis_results.png', dpi=300, bbox_inches='tight')
print("\nVisualizations saved as 'pca_analysis_results.png'")
plt.show()

print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)
print("\nNote: You're modeling CHANGES in log prices using CHANGES in predictors.")
print("This is appropriate for non-stationary time series data.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.tsa.ar_model import AutoReg
import warnings
warnings.filterwarnings('ignore')

# ========================================
# 1. Baseline: AR(2) Model
# ========================================

print("=" * 80)
print("BASELINE: AR(2) MODEL")
print("=" * 80)

# Prepare data for AR model
y_series = data['real_prices_diff'].dropna()

# Split data (80-20, time-ordered)
train_size = int(len(y_series) * 0.8)
y_train_ar = y_series.iloc[:train_size]
y_test_ar = y_series.iloc[train_size:]

# Fit AR(2) model
ar2_model = AutoReg(y_train_ar, lags=2, trend='n').fit()

# Predictions
y_test_pred_ar2 = ar2_model.predict(start=len(y_train_ar), end=len(y_series)-1)

# Metrics
ar2_r2 = r2_score(y_test_ar, y_test_pred_ar2)
ar2_rmse = np.sqrt(mean_squared_error(y_test_ar, y_test_pred_ar2))
ar2_mae = mean_absolute_error(y_test_ar, y_test_pred_ar2)

print(f"\nAR(2) Performance:")
print(f"  Test R²: {ar2_r2:.4f}")
print(f"  Test RMSE: {ar2_rmse:.4f}")
print(f"  Test MAE: {ar2_mae:.4f}")
print(f"\n→ This is the benchmark to beat!")

# ========================================
# 2. Advanced Feature Engineering
# ========================================

def create_advanced_features(data, max_lag=6):
    """
    Create comprehensive feature set including:
    - Lagged features
    - Autoregressive terms (AR)
    - Rolling statistics
    - Interaction terms
    """
    df = data.copy()
    
    # Get feature columns
    feature_cols = [col for col in df.columns if col != 'real_prices_diff']
    
    # === 1. Autoregressive terms (critical for beating AR(2)) ===
    df['price_lag1'] = df['real_prices_diff'].shift(1)
    df['price_lag2'] = df['real_prices_diff'].shift(2)
    df['price_lag3'] = df['real_prices_diff'].shift(3)
    
    # === 2. Rolling statistics of price ===
    df['price_rolling_mean_3'] = df['real_prices_diff'].shift(1).rolling(3).mean()
    df['price_rolling_std_3'] = df['real_prices_diff'].shift(1).rolling(3).std()
    df['price_rolling_mean_6'] = df['real_prices_diff'].shift(1).rolling(6).mean()
    
    # === 3. Lagged exogenous variables ===
    for col in feature_cols:
        df[f'{col}_lag1'] = df[col].shift(1)
        df[f'{col}_lag3'] = df[col].shift(3)
        df[f'{col}_lag6'] = df[col].shift(6)
        
        # Rolling averages of key variables
        if col in ['loans_diff', 'mortgage_rate_diff', 'unemployment_rate_diff']:
            df[f'{col}_rolling_mean_3'] = df[col].shift(1).rolling(3).mean()
    
    # === 4. Interaction terms (key relationships) ===
    # Mortgage rate * income (affordability)
    df['mortgage_income_interact'] = (
        df['mortgage_rate_diff_lag1'] * df['median_house_income_diff_lag1']
    )
    
    # Supply * vacancy (market tightness)
    df['supply_vacancy_interact'] = (
        df['monthly_supply_homes_diff_lag1'] * df['vacancy_rate_diff_lag1']
    )
    
    # Loans * wealth (credit conditions)
    df['loans_wealth_interact'] = (
        df['loans_diff_lag1'] * df['net_wealth_diff_lag1']
    )
    
    # === 5. Momentum indicators ===
    # Price momentum (change in change)
    df['price_momentum'] = df['price_lag1'] - df['price_lag2']
    
    # === 6. Current period features ===
    for col in feature_cols:
        df[f'{col}_current'] = df[col]
    
    return df.dropna()

print("\n" + "=" * 80)
print("CREATING ADVANCED FEATURES...")
print("=" * 80)

data_advanced = create_advanced_features(data, max_lag=6)

print(f"\nOriginal features: {data.shape[1]}")
print(f"Advanced features: {data_advanced.shape[1]}")
print(f"Rows after lagging: {len(data_advanced)}")

# ========================================
# 3. Test Multiple ML Models
# ========================================

# Prepare data
X = data_advanced.drop('real_prices_diff', axis=1)
y = data_advanced['real_prices_diff']

# Time-ordered split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False, random_state=42
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Define models to test
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=300,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    ),
    'Ridge (L2)': Ridge(alpha=1.0),
    'Lasso (L1)': Lasso(alpha=0.001, max_iter=10000)
}

print("\n" + "=" * 80)
print("TRAINING MODELS...")
print("=" * 80)

results = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    # Check if beats AR(2)
    beats_ar2 = "✓" if test_r2 > ar2_r2 else "✗"
    improvement = test_r2 - ar2_r2
    
    results.append({
        'Model': name,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'Test RMSE': test_rmse,
        'Test MAE': test_mae,
        'Overfit': train_r2 - test_r2,
        'Beats AR(2)': beats_ar2,
        'Improvement': improvement
    })
    
    print(f"  Test R²: {test_r2:.4f} {beats_ar2}")
    print(f"  Improvement over AR(2): {improvement:+.4f}")

# Add AR(2) to results for comparison
results.insert(0, {
    'Model': 'AR(2) Baseline',
    'Train R²': np.nan,
    'Test R²': ar2_r2,
    'Test RMSE': ar2_rmse,
    'Test MAE': ar2_mae,
    'Overfit': np.nan,
    'Beats AR(2)': '—',
    'Improvement': 0.0
})

results_df = pd.DataFrame(results)

print("\n" + "=" * 80)
print("MODEL COMPARISON")
print("=" * 80)
print(results_df.to_string(index=False))

# ========================================
# 4. Best Model Analysis
# ========================================

# Find best model (excluding AR(2))
best_idx = results_df[results_df['Model'] != 'AR(2) Baseline']['Test R²'].idxmax()
best_model_name = results_df.loc[best_idx, 'Model']
best_model = models[best_model_name]

print("\n" + "=" * 80)
print(f"BEST MODEL: {best_model_name}")
print("=" * 80)

best_model.fit(X_train, y_train)
y_test_pred_best = best_model.predict(X_test)

# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 Most Important Features:")
    print(feature_importance.head(15).to_string(index=False))
    
    # Check if AR terms are important
    ar_features = feature_importance[feature_importance['feature'].str.contains('price_lag')]
    print("\n→ Autoregressive Feature Importance:")
    print(ar_features.to_string(index=False))

# ========================================
# 5. Visualizations
# ========================================

fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# Plot 1: Model Comparison
ax1 = fig.add_subplot(gs[0, :])
models_sorted = results_df.sort_values('Test R²', ascending=True)
colors = ['red' if m == 'AR(2) Baseline' else 'steelblue' for m in models_sorted['Model']]
bars = ax1.barh(models_sorted['Model'], models_sorted['Test R²'], color=colors, edgecolor='black')
ax1.axvline(x=ar2_r2, color='red', linestyle='--', linewidth=2, label=f'AR(2) Baseline (R²={ar2_r2:.3f})')
ax1.set_xlabel('Test R²', fontsize=12, fontweight='bold')
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='x')

for i, (bar, val) in enumerate(zip(bars, models_sorted['Test R²'])):
    ax1.text(val, bar.get_y() + bar.get_height()/2, f'  {val:.4f}',
             va='center', fontsize=9, fontweight='bold')

# Plot 2: Actual vs Predicted (Best Model)
ax2 = fig.add_subplot(gs[1, 0])
ax2.scatter(y_test, y_test_pred_best, alpha=0.6, edgecolors='k', linewidth=0.5, s=50)
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
ax2.set_xlabel('Actual Price Change', fontsize=11)
ax2.set_ylabel('Predicted Price Change', fontsize=11)
ax2.set_title(f'{best_model_name}: Actual vs Predicted\nR² = {r2_score(y_test, y_test_pred_best):.4f}', 
              fontsize=11, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Residuals (Best Model)
ax3 = fig.add_subplot(gs[1, 1])
residuals = y_test - y_test_pred_best
ax3.scatter(y_test_pred_best, residuals, alpha=0.6, edgecolors='k', linewidth=0.5, s=50)
ax3.axhline(y=0, color='r', linestyle='--', lw=2)
ax3.set_xlabel('Predicted Price Change', fontsize=11)
ax3.set_ylabel('Residuals', fontsize=11)
ax3.set_title(f'{best_model_name}: Residual Plot', fontsize=11, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Feature Importance (if available)
if hasattr(best_model, 'feature_importances_'):
    ax4 = fig.add_subplot(gs[1, 2])
    top_features = feature_importance.head(15)
    
    # Color code: AR features vs exogenous
    colors_feat = ['darkred' if 'price_lag' in f or 'price_rolling' in f or 'price_momentum' in f
                   else 'steelblue' for f in top_features['feature']]
    
    ax4.barh(range(len(top_features)), top_features['importance'], color=colors_feat, edgecolor='black')
    ax4.set_yticks(range(len(top_features)))
    ax4.set_yticklabels(top_features['feature'], fontsize=8)
    ax4.set_xlabel('Importance', fontsize=11)
    ax4.set_title(f'Top 15 Features\n(Red = AR features)', fontsize=11, fontweight='bold')
    ax4.invert_yaxis()
    ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# ========================================
# 6. Interpretation & Recommendations
# ========================================

print("\n" + "=" * 80)
print("INTERPRETATION & NEXT STEPS")
print("=" * 80)

best_r2 = results_df.loc[best_idx, 'Test R²']
improvement = best_r2 - ar2_r2

print(f"\n1. DID WE BEAT AR(2)?")
if improvement > 0.01:
    print(f"   ✓ YES! Improvement of {improvement:.4f} ({improvement/abs(ar2_r2)*100:.1f}%)")
    print(f"   → {best_model_name} with exogenous features beats pure autoregression")
elif improvement > 0:
    print(f"   ⚠ MARGINAL. Improvement of only {improvement:.4f}")
    print(f"   → External features add minimal value beyond AR(2)")
else:
    print(f"   ✗ NO. Performance is {improvement:.4f} worse than AR(2)")
    print(f"   → Price changes are mainly driven by their own history")

print(f"\n2. KEY INSIGHTS:")
if hasattr(best_model, 'feature_importances_'):
    ar_total_importance = ar_features['importance'].sum()
    print(f"   - Autoregressive features account for {ar_total_importance:.1%} of importance")
    
    exog_important = feature_importance[~feature_importance['feature'].str.contains('price_')]
    if len(exog_important) > 0:
        top_exog = exog_important.iloc[0]
        print(f"   - Most important external factor: {top_exog['feature']} ({top_exog['importance']:.3f})")

print(f"\n3. RECOMMENDATIONS:")
if improvement > 0.05:
    print("   → Use this ML model for forecasting - it clearly beats AR(2)")
    print("   → External economic factors provide meaningful signal")
elif improvement > 0:
    print("   → Consider ensemble: combine AR(2) with ML predictions")
    print("   → External factors help but marginally")
else:
    print("   → Stick with AR(2) - it's simpler and performs better")
    print("   → Try: different feature engineering, longer lags, or regime-switching models")

print("\n" + "=" * 80)

In [ ]:
"""
Advanced Machine Learning Pipeline for Time Series Forecasting
Beats AR(2) baseline by incorporating economic features and ensemble methods
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.ar_model import AutoReg
import warnings
warnings.filterwarnings('ignore')

# ================================================================================
# 1. LOAD AND PREPARE DATA
# ================================================================================

def load_data(filepath):
    """Load your dataset - modify path as needed"""
    df = pd.read_csv(filepath)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    return df

def create_advanced_features(df, target_col='price'):
    """
    Create comprehensive feature set including:
    - Autoregressive lags
    - Rolling statistics
    - Differences and momentum
    - Interaction terms
    """
    df = df.copy()
    
    # 1. Autoregressive features (lags 1-6)
    for lag in range(1, 7):
        df[f'{target_col}_lag{lag}'] = df[target_col].shift(lag)
    
    # 2. Rolling statistics
    for window in [3, 6, 12]:
        df[f'{target_col}_rolling_mean_{window}'] = df[target_col].shift(1).rolling(window).mean()
        df[f'{target_col}_rolling_std_{window}'] = df[target_col].shift(1).rolling(window).std()
        df[f'{target_col}_rolling_min_{window}'] = df[target_col].shift(1).rolling(window).min()
        df[f'{target_col}_rolling_max_{window}'] = df[target_col].shift(1).rolling(window).max()
    
    # 3. Momentum and differences
    df[f'{target_col}_momentum'] = df[target_col].shift(1) - df[target_col].shift(4)
    df[f'{target_col}_diff'] = df[target_col].diff()
    df[f'{target_col}_pct_change'] = df[target_col].pct_change()
    
    # 4. Exogenous variable features (adjust column names to match your data)
    exog_cols = [col for col in df.columns if col not in ['date', target_col] and not col.endswith(tuple([f'_lag{i}' for i in range(1,7)]))]
    
    for col in exog_cols:
        if df[col].dtype in ['float64', 'int64']:
            # Current and lagged values
            df[f'{col}_current'] = df[col]
            for lag in [1, 2, 3]:
                df[f'{col}_lag{lag}'] = df[col].shift(lag)
            
            # Differences
            df[f'{col}_diff'] = df[col].diff()
            df[f'{col}_diff_lag1'] = df[f'{col}_diff'].shift(1)
            df[f'{col}_diff_lag3'] = df[f'{col}_diff'].shift(3)
            
            # Rolling statistics
            df[f'{col}_diff_rolling_mean_3'] = df[f'{col}_diff'].shift(1).rolling(3).mean()
    
    # 5. Interaction terms (example - adjust based on your domain knowledge)
    if 'loans' in exog_cols and 'wealth' in exog_cols:
        df['loans_wealth_interact'] = df['loans'] * df['wealth']
    
    # 6. Seasonal features (if you have date)
    if 'date' in df.columns:
        df['month'] = df['date'].dt.month
        df['quarter'] = df['date'].dt.quarter
        df['year'] = df['date'].dt.year
    
    return df

# ================================================================================
# 2. BASELINE AR(2) MODEL
# ================================================================================

def train_ar2_baseline(y_train, y_test):
    """Train baseline AR(2) model for comparison"""
    ar2_model = AutoReg(y_train, lags=2).fit()
    
    # Forecast on test set
    predictions = []
    history = list(y_train)
    
    for i in range(len(y_test)):
        lag1 = history[-1]
        lag2 = history[-2]
        pred = ar2_model.params[0] + ar2_model.params[1]*lag1 + ar2_model.params[2]*lag2
        predictions.append(pred)
        history.append(y_test.iloc[i])
    
    predictions = np.array(predictions)
    
    r2 = r2_score(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    
    return {
        'model': ar2_model,
        'predictions': predictions,
        'r2': r2,
        'rmse': rmse,
        'mae': mae
    }

# ================================================================================
# 3. ADVANCED ML MODELS WITH TUNING
# ================================================================================

def train_random_forest_tuned(X_train, y_train, X_test, y_test, n_iter=30):
    """Train Random Forest with hyperparameter tuning"""
    print("Training Random Forest with hyperparameter search...")
    
    param_dist = {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [10, 15, 20, 25, None],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4, 6],
        'max_features': ['sqrt', 'log2', 0.3, 0.5]
    }
    
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    rf_search = RandomizedSearchCV(
        rf, param_dist, n_iter=n_iter, 
        cv=tscv, scoring='r2', 
        n_jobs=-1, random_state=42, verbose=1
    )
    
    rf_search.fit(X_train, y_train)
    
    best_model = rf_search.best_estimator_
    
    train_pred = best_model.predict(X_train)
    test_pred = best_model.predict(X_test)
    
    return {
        'model': best_model,
        'best_params': rf_search.best_params_,
        'train_r2': r2_score(y_train, train_pred),
        'test_r2': r2_score(y_test, test_pred),
        'test_rmse': np.sqrt(mean_squared_error(y_test, test_pred)),
        'test_mae': mean_absolute_error(y_test, test_pred),
        'predictions': test_pred
    }

def train_gradient_boosting_tuned(X_train, y_train, X_test, y_test):
    """Train Gradient Boosting with regularization to prevent overfitting"""
    print("Training Gradient Boosting with regularization...")
    
    param_dist = {
        'n_estimators': [50, 100, 150, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [2, 3, 4, 5],
        'min_samples_split': [5, 10, 20],
        'min_samples_leaf': [2, 4, 6],
        'subsample': [0.6, 0.8, 1.0]
    }
    
    gb = GradientBoostingRegressor(random_state=42)
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    gb_search = RandomizedSearchCV(
        gb, param_dist, n_iter=30,
        cv=tscv, scoring='r2',
        n_jobs=-1, random_state=42, verbose=1
    )
    
    gb_search.fit(X_train, y_train)
    
    best_model = gb_search.best_estimator_
    
    train_pred = best_model.predict(X_train)
    test_pred = best_model.predict(X_test)
    
    return {
        'model': best_model,
        'best_params': gb_search.best_params_,
        'train_r2': r2_score(y_train, train_pred),
        'test_r2': r2_score(y_test, test_pred),
        'test_rmse': np.sqrt(mean_squared_error(y_test, test_pred)),
        'test_mae': mean_absolute_error(y_test, test_pred),
        'predictions': test_pred
    }

def train_ensemble(rf_model, gb_model, X_train, y_train, X_test, y_test):
    """Create ensemble of best models"""
    print("Creating ensemble model...")
    
    ensemble = VotingRegressor([
        ('rf', rf_model),
        ('gb', gb_model)
    ])
    
    ensemble.fit(X_train, y_train)
    
    train_pred = ensemble.predict(X_train)
    test_pred = ensemble.predict(X_test)
    
    return {
        'model': ensemble,
        'train_r2': r2_score(y_train, train_pred),
        'test_r2': r2_score(y_test, test_pred),
        'test_rmse': np.sqrt(mean_squared_error(y_test, test_pred)),
        'test_mae': mean_absolute_error(y_test, test_pred),
        'predictions': test_pred
    }

# ================================================================================
# 4. FEATURE SELECTION
# ================================================================================

def select_important_features(model, X_train, X_test, threshold='median'):
    """Select most important features using trained model"""
    selector = SelectFromModel(model, threshold=threshold, prefit=True)
    
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    selected_features = X_train.columns[selector.get_support()].tolist()
    
    print(f"\nFeature selection: {len(selected_features)} features selected from {X_train.shape[1]}")
    
    return X_train_selected, X_test_selected, selected_features

# ================================================================================
# 5. VISUALIZATION AND REPORTING
# ================================================================================

def plot_feature_importance(model, feature_names, top_n=20):
    """Plot feature importance for tree-based models"""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'estimators_'):  # For ensemble models
        importances = model.estimators_[0].feature_importances_
    else:
        print("Model doesn't have feature importances")
        return
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).head(top_n)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(importance_df)), importance_df['importance'])
    plt.yticks(range(len(importance_df)), importance_df['feature'])
    plt.xlabel('Importance')
    plt.title(f'Top {top_n} Most Important Features')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    return importance_df

def plot_predictions(y_test, predictions_dict, dates=None):
    """Plot actual vs predicted values for all models"""
    plt.figure(figsize=(14, 6))
    
    if dates is not None:
        x = dates
        plt.xlabel('Date')
    else:
        x = range(len(y_test))
        plt.xlabel('Time Step')
    
    plt.plot(x, y_test.values, label='Actual', linewidth=2, color='black', alpha=0.7)
    
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, (name, pred) in enumerate(predictions_dict.items()):
        plt.plot(x, pred, label=name, linewidth=1.5, alpha=0.6, color=colors[i % len(colors)])
    
    plt.ylabel('Target Value')
    plt.title('Model Predictions vs Actual Values')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def create_comparison_table(results_dict, ar2_baseline):
    """Create comprehensive model comparison table"""
    comparison = []
    
    # Add AR(2) baseline
    comparison.append({
        'Model': 'AR(2) Baseline',
        'Train R²': np.nan,
        'Test R²': ar2_baseline['r2'],
        'Test RMSE': ar2_baseline['rmse'],
        'Test MAE': ar2_baseline['mae'],
        'Overfit': np.nan,
        'vs Baseline': 0.0
    })
    
    # Add ML models
    for name, results in results_dict.items():
        comparison.append({
            'Model': name,
            'Train R²': results.get('train_r2', np.nan),
            'Test R²': results['test_r2'],
            'Test RMSE': results['test_rmse'],
            'Test MAE': results['test_mae'],
            'Overfit': results.get('train_r2', np.nan) - results['test_r2'],
            'vs Baseline': results['test_r2'] - ar2_baseline['r2']
        })
    
    df = pd.DataFrame(comparison)
    df = df.sort_values('Test R²', ascending=False)
    
    return df

# ================================================================================
# 6. MAIN PIPELINE
# ================================================================================

def main(filepath, target_col='price', test_size=0.2):
    """
    Main pipeline that orchestrates the entire analysis
    
    Parameters:
    -----------
    filepath : str
        Path to your CSV file
    target_col : str
        Name of target variable column
    test_size : float
        Proportion of data to use for testing
    """
    
    print("="*80)
    print("ADVANCED ML TIME SERIES FORECASTING PIPELINE")
    print("="*80)
    
    # Load data
    print("\n1. Loading data...")
    df = load_data(filepath)
    print(f"   Loaded {len(df)} rows, {len(df.columns)} columns")
    
    # Create features
    print("\n2. Creating advanced features...")
    df_features = create_advanced_features(df, target_col)
    
    # Remove rows with NaN (from lagging)
    df_features = df_features.dropna()
    print(f"   Created {len(df_features.columns) - len(df.columns)} new features")
    print(f"   Rows after lagging: {len(df_features)}")
    
    # Prepare train/test split
    split_idx = int(len(df_features) * (1 - test_size))
    
    feature_cols = [col for col in df_features.columns if col not in ['date', target_col]]
    
    X = df_features[feature_cols]
    y = df_features[target_col]
    
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    dates_test = df_features['date'].iloc[split_idx:] if 'date' in df_features.columns else None
    
    print(f"   Training samples: {len(X_train)}")
    print(f"   Test samples: {len(X_test)}")
    
    # Train baseline AR(2)
    print("\n3. Training AR(2) baseline...")
    ar2_results = train_ar2_baseline(y_train, y_test)
    print(f"   Test R²: {ar2_results['r2']:.4f}")
    print(f"   Test RMSE: {ar2_results['rmse']:.4f}")
    
    # Train ML models
    print("\n4. Training advanced ML models...")
    print("-"*80)
    
    rf_results = train_random_forest_tuned(X_train, y_train, X_test, y_test, n_iter=20)
    print(f"   Random Forest - Test R²: {rf_results['test_r2']:.4f}")
    print(f"   Best params: {rf_results['best_params']}")
    
    gb_results = train_gradient_boosting_tuned(X_train, y_train, X_test, y_test)
    print(f"   Gradient Boosting - Test R²: {gb_results['test_r2']:.4f}")
    print(f"   Best params: {gb_results['best_params']}")
    
    # Create ensemble
    print("\n5. Creating ensemble model...")
    ensemble_results = train_ensemble(
        rf_results['model'], 
        gb_results['model'],
        X_train, y_train, X_test, y_test
    )
    print(f"   Ensemble - Test R²: {ensemble_results['test_r2']:.4f}")
    
    # Model comparison
    print("\n6. Model Comparison")
    print("="*80)
    
    results_dict = {
        'Random Forest': rf_results,
        'Gradient Boosting': gb_results,
        'Ensemble': ensemble_results
    }
    
    comparison_df = create_comparison_table(results_dict, ar2_results)
    print(comparison_df.to_string(index=False))
    
    # Feature importance
    print("\n7. Feature Importance Analysis")
    print("="*80)
    
    best_model_name = comparison_df.iloc[1]['Model']  # Second row (first is baseline)
    best_model = results_dict[best_model_name]['model']
    
    importance_df = plot_feature_importance(best_model, feature_cols, top_n=20)
    print("\nTop 10 features:")
    print(importance_df.head(10).to_string(index=False))
    
    # Predictions plot
    print("\n8. Generating prediction plots...")
    predictions_dict = {
        'AR(2)': ar2_results['predictions'],
        'Random Forest': rf_results['predictions'],
        'Gradient Boosting': gb_results['predictions'],
        'Ensemble': ensemble_results['predictions']
    }
    
    plot_predictions(y_test, predictions_dict, dates_test)
    
    # Final recommendations
    print("\n" + "="*80)
    print("RECOMMENDATIONS")
    print("="*80)
    
    best_r2 = comparison_df.iloc[1]['Test R²']
    improvement = comparison_df.iloc[1]['vs Baseline']
    
    if best_r2 > ar2_results['r2']:
        print(f"✓ Use {best_model_name} for forecasting")
        print(f"  Improvement over AR(2): {improvement:.4f} ({improvement/ar2_results['r2']*100:.1f}%)")
        print(f"  Test R²: {best_r2:.4f}")
    else:
        print("✗ ML models did not beat baseline - stick with AR(2)")
    
    print("\nNext steps:")
    print("  1. Monitor model performance over time")
    print("  2. Retrain quarterly with new data")
    print("  3. Set up prediction intervals for risk management")
    
    return {
        'models': results_dict,
        'comparison': comparison_df,
        'feature_importance': importance_df,
        'test_data': (X_test, y_test),
        'best_model': best_model
    }

# ================================================================================
# USAGE EXAMPLE
# ================================================================================

if __name__ == "__main__":
    # Modify these parameters for your data
    FILEPATH = 'your_data.csv'  # Change to your file path
    TARGET_COL = 'price'        # Change to your target column name
    TEST_SIZE = 0.2             # 20% test set
    
    # Run pipeline
    results = main(
        filepath=FILEPATH,
        target_col=TARGET_COL,
        test_size=TEST_SIZE
    )
    
    # Access results
    best_model = results['best_model']
    comparison_table = results['comparison']
    feature_importance = results['feature_importance']
    
    # Make future predictions (example)
    # X_future = create_features_for_future_data(...)
    # predictions = best_model.predict(X_future)

In [ ]:
from sklearn.ensemble import VotingRegressor

ensemble = VotingRegressor([
    ('rf', tuned_rf),
    ('gb', tuned_gb)
])